<center>
<h2>Introduction au Webscraping</h2>
<h3>Principes fondamentaux et mise en pratique</h3>
</center>
<br/>

## Introduction

### Objectifs de la présentation

* Introduire les concepts et éléments techniques fondamentaux du _webscraping_.
* Présenter les étapes-clefs d'une tâche _webscraping_ et leur articulation.
* Illustrer la mise en place d'un webscraping avec différents outils, en Python.

### Notions fondamentales du _webscraping_

* Le _webscraping_ est une activité de collecte (et généralement d'extraction) automatisée de données en ligne.
  * Il s'agit d'accéder de manière automatisée à des pages web (et autres ressources en ligne) en vue d'en extraire
    certains contenus.
    
  * Une première étape est donc d'accéder à la page "contenant" l'information que l'on cherche à extraire.
  
  * La seconde étape consiste alors à extraire et structurer l'information contenue dans cette page.


* Il est donc nécessaire de savoir sur quelle(s) page(s) récupérer l'information.
  * Dans certains cas, on sait exactement quelle(s) page(s) on souhaite consulter : on peut alors simplement en
    établir la liste et scraper celles-ci. _E.g. : scraper une liste de pages d'accueils de journaux pour extraire
    le titre des articles en "Une"._
   
  * Dans d'autres cas, on peut établir des règles de navigation automatisée, afin de "découvrir" les pages à scraper
    à partir d'un ou plusieurs points d'entrée. _E.g. : scraper la page d'accueil du journal Le Monde, trouver tous
    les liens vers des articles puis scraper ceux-ci afin d'en récupérer le titre, l'auteur et (si possible) le texte._
    
  * La navigation automatisée le long des liens extraits à partir d'un point d'entrée est appellée _web crawling_.


* Il faut ensuite définir des règles d'extraction de l'information.
  * Dans certains cas, on connaît la structure des pages que l'on scrape : on peut alors définir assez finement des
    règles de sélection de l'information, afin de sélectionner exactement ce que l'on veut, de manière déjà structurée.
    _E.g. : tous les articles du site Le Monde suivent un modèle similaire, il est donc possible d'étudier celui-ci
    pour définir des règles d'extraction précises, valides pour tous les articles ciblés._
  
  * Dans d'autres cas, l'hétérogénéité des pages que l'on scrape rend très couteuse la définition de telles règles. Des
    règles plus générales, qui risquent cependant d'être moins précises, peuvent alors être utilisées. _E.g. : si l'on
    cible tous les articles référencés sur Google News, la diversité des médias rend difficile le scraping "fin" des
    pages. On va alors définir des règles assez générales pour identifier le texte de l'article, quitte à définir des
    règles plus fines pour quelques sites jugés majeurs._

## 1. Éléments techniques fondamentaux 

### 1.1 Requêter une page web

* URL d'une ressource
  * Uniform Resource Locator, ou URL, est une norme d'adressage des ressources sur internet.
  
  * Une URL spécifie (_a minima_) un protocole d'accès à la ressource et la localisation de celle-ci.
    Cette localisation comprend notamment un nom de domaine (ou une adresse IP). Il est également possible
    de spécifier un certain nombre d'options.
    
  * Exemple : `https://duckduckgo.com/?q=webscraping`. Il s'agit de requêter DuckDuckGo à l'aide du protocole HTTPS
    (cf. infra), et plus spécifiquement d'effectuer _via_ le service fourni par cet excellent moteur de recherche
    la recherche du terme "webscraping".
  
  * Afin de collecter une page web, on va donc effectuer une requête vers son URL, en spécifiant éventuellement
    certains paramètres.


* Requêtes HTTP
  * Hyper Text Transfer Protocol, ou HTTP, est un protocole de communication entre un client (e.g. un navigateur web)
    et un serveur (e.g. le serveur hébergeant un site web) développé pour le web.
    * Une variante communément rencontrée est HTTPS ; le S abrège "Secured" et désigne le recours conjoint à d'autres
      protocoles (SSL ou TLS) pour sécuriser l'échange (authentification du serveur, chiffrement des données...).
    * Il existe d'autres protocoles permettant d'échanger des données sur internet ; ils ne relèvent cependant pas du
      _webscraping_ à proprement parler, aussi ne les mentionnerons-nous pas.
    
  * HTTP permet d'effectuer plusieurs types de requête, dont la principale est GET, qui permet de demander une ressource.
    D'autres requêtes, dont POST (ou PUT), peuvent être utiles dans certains cas lorsque l'on scrape un site (nous y
    reviendrons).

  * À l'issue d'une requête HTTP, un code d'état est assigné à son résultat. Celui-ci permet de caractériser le
    succès de la requête, ou de spécifier la nature d'un échec le cas échéant. Il est donc très utile pour ajuster
    le comportement d'un programme de _webscraping_ suivant le type d'erreur qu'il peut rencontrer. Voici quelques
    codes qu'il est utile de connaître :
    * Le code 200 indique le succès de la requête. (_on peut donc exploiter la ressource_)
    * Les codes 301 et 302 indiquent une redirection (permanente ou temporaire) vers une autre URL à partir
      de celle requêtée. (_on va donc vérifier que la ressource correspond à celle recherchée_)
    * Le code 403 indique que l'accès à la ressource demandée a été refusé. (_on va donc abandonner la ressource, ou
      réessayer plus tard s'il s'agit d'une interdiction liée à une restriction du nombre de requêtes authorisées_)
    * Le code 404 indique que la ressource demandée n'a pas été trouvée (_il faut donc abandonner la ressource, ou
      rechercher sa nouvelle adresse si elle existe_).
    * Le code 443 indique un blocage de la requête par le Proxy (_il faut donc réviser la configuration de l'accès à
      celui-ci, potentiellement (dans le cadre institutionnel) en concertation avec les équipes informatiques locales_).
    * Les codes 500 et 503 indiquent des erreurs au niveau du serveur (_on peut donc tenter de requêter à nouveau, tout
      de suite ou plus tard_).



* Requêter une page web en Python.
  * `Requests` est une librairie tierce pour Python, construite au-dessus de la librairie standard `urllib3`.
    Elle est documentée <a href="http://docs.python-requests.org/en/master/">ici</a>.
  * Conçue pour être facile d'accès, elle est idéale pour cette présentation ; chacun est ensuite libre de
    lui préférer la librairie standard, ou une autre librairie tierce. Dans tous les cas, la syntaxe change
    (un peu), mais le principe reste le même, et les mécanismes sous-jacents sont strictement identiques.
  * Pour les adeptes de R, `RCurl` est plus ou moins l'équivalent de `urllib3`, tandis que `httr` semble un
    peu plus accesible et que `rvest` est plus spécifiquement adapté au _webscraping_. Pour une documentation
    des packages existants pour faire des requêtes (et extraire de l'information des pages web), consulter
    <a href="https://cran.r-project.org/web/views/WebTechnologies.html">cette page</a>.

In [ ]:
import requests

In [ ]:
# Requête GET sur l'url correspondant à la page d'accueil de l'Insee.
response = requests.get('https://insee.fr/fr/accueil')

print('Url: %s' % response.url)
print('Statut: %s' % response.status_code)

** Question : Génial, mais alors qu'est-ce qu'on a récupéré ? **

In [ ]:
print(response.text)

** Réponse : le code source de la page d'accueil de l'INSEE. **

### 1.2 Le HTML, code source des pages web.

#### Qu'est-ce que le HTML ?

* Hyper Text Markup Language (HTML) est le langage utilisé pour écrire des pages web. Il s'agit d'un langage de balisage 
  (comme LaTeX et XML, notamment) : des balises (de forme `<balise>`) structurent le contenu de la page et dessinent une 
  arborescence de contenus.
    

* La page visualisée dans un navigateur web est générée localement à partir du code HTML.
  * La connexion a un site entraîne la requête et le téléchargement du HTML de la page (et de ressources associées).
  * Le rôle du navigateur web est alors de "rendre" le HTML, c'est-à-dire de générer le contenu que l'on visualise
    et avec lequel on est amené à intéragir.
  * Note : les interactions proviennent de la définition d'événements pouvant (ou non) entraîner l'exécution de
    requêtes supplémentaires (asynchrones) ou la modification de propriétés associées à des balises déjà chargées.
  * En particulier, il est possible d'intégrer nativement du code en JavaScript (langage de programmation interprété)
    dans la page, dont l'interprétation revient au navigateur. Pour d'autres technologies (et dans certains cas pour
    du JavaScript), les opérations ont lieu du côté du site.
  * Afin de spécifier les paramètres de style des éléments (taille, couleur, etc.), il est également d'usage de
    recourir au langage CSS (Cascade Style Sheet), souvent chargé depuis un ou plusieurs fichiers `.css`. Ce n'est
    cependant (en général) pas cela qui nous intéresse dans le cadre du scraping, et nous laisserons donc cet aspect
    de côté.


* Pour bien le comprendre, on peut recourir à un outil intégré à certains navigateurs (en particulier,
  Mozilla Firefox) : l'inspecteur d'éléments. Celui-ci permet de consulter le html de la page actuellement
  rendue, et de le modifier (localement !), altérant ainsi la page rendue dans le navigateur.

#### Éléments fondamentaux de structure d'une page HTML

* La structuration du HTML se fait autour de balises.
  * La plupart des balises fonctionnent par paire : une balise ouvrante `<balise>` et une balise fermante `</balise>`.
  
  * Quelques balises sont auto-fermantes ; il est alors de bon aloi de les écrirer `<balise/>`.
    La plus typique est la balise de retour à la ligne : `<br/>` (qui connaît des variantes suivant les conventions
    de la personne (ou du programme) écrivant le html.
    
  * Une balise possède un type, qui est le premier mot de la balise ouvrante (et le seul de la balise fermante).
    Elle peut aussi être dotée d'attributs, écrits dans la balise ouvrante, au format `attribut="valeur"`.
    Quelques exemples : 
    * `<div id="toto">` est une `div` (division) dont l'attribut `id` est `toto`.
    * `<p style="font-size: 18px;">` est un `p` (paragraphe) dont le texte va être rendu dans une police de 18 pixels.
    * `<a href="https://insee.fr/">` est un `a` (lien) pointant vers https://insee.fr/.

  * Il existe un grand nombre de balises et d'attributs "standards", qui pour certains impliquent par soi un
    comportement spécifique (e.g. `<ul>` définit une liste sans ordre (_unordered list_), si bien que chaque
    balise `<li>` va générer une puce devant son contenu). Il est également possible de définir arbitrairement
    des balises ou attributs, en respectant ou non les conventions préconisées par le
    <a href="https://www.w3schools.com/default.asp">W3C</a> (World Wide Web Consortium).

* Trois balises fournissent la structure essentielle de toute page html :
  * `<html>` : balise principale, contenant l'intégralité de la page.
  * `<header>` : espace où sont notamment renseignées des méta-informations, et où sont souvent chargées ou
    définies des ressources "non affichées" (tels des scripts ou règles de style graphique).
  * `<body>` : espace où se trouve le contenu visible de la page ; il est également fréquent d'y rencontrer
    des scripts définissant des actions spécifiques.
  

* Quelques types de balises essentiels :
  * `<h1>` (et `<h2>`, etc.) : balises définissant des styles de titre.
  * `<div>` : "division" ; cette balise est l'une des plus usuelles et diverses.
  * `<p>` : "paragraphe", supposé encadrer du texte.
  * `<a>` : "lien" (_anchor_), pouvant pointer sur une section de la page ou vers une autre adresse.
  * `<img/>` : balise auto-fermante permettant de charger un image dans la page.
  * `<script>` : script JavaScript, pouvant parfois contenir des informations d'intérêt (on y reviendra).
  * `<meta/>` : balise auto-fermante indiquant une méta-information, présente dans le `<header>`


* Quelques attributs de balises essentiels :
  * `id` : identifiant (nommé) ne pouvant être partagé avec une autre balise.
  * `class` : identifiant (nommé) pouvant être commun à plusieurs balises.
  * `href` : adresse vers laquelle pointe la balise (typiquement: `<a href="url">texte du lien</a>`)
  * `src` : source d'une ressource chargée par la balise (e.g. `<img src="image.png" />`)

In [ ]:
# Écrivons une page html basique, qui dit "Hello World".
html = """<!DOCTYPE html>
<html>
    <head>
        <title>Hello World</title>
        <meta author="Paul Andrey"/>
    </head>
    <body>
        <center>Hello World!</center>
    </body>
</html>
"""
with open('hello.html', mode='w', encoding='utf-8') as html_file:
    html_file.write(html)

In [ ]:
# Par la suite, nous rendrons le HTML directement dans le notebook, comme suit :
from IPython.core.display import HTML
HTML(html)

<center>**Nous savons donc requêter le HTML source d'une page dont l'URL est connue, et savons que ce HTML est structuré.<br/>
Mais en pratique, comment extraire l'information qu'il contient ?**</center>

<br/>
## 2. Extraire de l'information d'une page HTML

### 2.1 Beautiful Soup

* Beautiful Soup (`bs4`) est une librairie tierce pour Python permettant de parser (i.e. analyser syntaxiquement) divers langages de balisage, et notamment le HTML. Elle s'appuie en partie sur la librairie standard `html`, et fournit une interface plutôt pratique.
  * La documentation de Beautiful Soup est consultable <a href="https://www.crummy.com/software/BeautifulSoup/">
    à cette adresse</a>.
  * D'autres parseurs existent, et aucun n'est parfait puisque le HTML rencontré "dans la vraie vie" est rarement exempt
    de "fautes" de syntaxe, auxquelles les parseurs s'ajustent diversement selon les cas.
  * Citons notamment la librairie `lxml`, qui permet notamment de travailler sur du XML ou du HTML ; nous ne
    l'utiliserons pas ici, cependant elle est assez aisée à prendre en main et présente une logique assez similaire
    avec celle de BeautifulSoup.


* BeautifulSoup est simple d'utilisation, et flexible :
  * On instancie un objet de classe `bs4.BeautifulSoup` à partir d'une donnée textuelle ou d'un fichier ouvert
    avec `open`, en spécifiant éventuellement le parseur à utiliser (`html.parser` pour html), quoi que celui-ci
    puisse normalement faire l'objet d'une détection automatique.
  * Il est alors possible de chercher une ou plusieurs balises au sein du HTML à partir de critères comme le type
    de balise, la valeur d'un ou plusieurs de ses attributs ou encore sa position relative à d'autres balises.

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
# Parsons le html de la page d'accueil de l'Insee.
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
# Cherchons la première balise <div> de la page.
first_div = soup.find('div')
first_div

In [ ]:
# On peut accéder aux attributs de la balise, sous forme de dictionaire.
first_div.attrs

In [ ]:
# On peut également accéder à son contenu, sous forme de liste de sous-balises.
first_div.contents

In [ ]:
# Une autre possibilité est d'itérer sur les sous-balises. Ici, on affiche leur type.
for tag in first_div.children:
    print(tag.name)

In [ ]:
# On peut aussi accéder au texte apparaissant à l'intérieur de la balise (et de ses sous-balises).
first_div.text

In [ ]:
# On peut maintenant chercher la seconde <div> de la page:
second_div = first_div.find_next_sibling()
second_div

In [ ]:
# Ou bien la première balise suivant la première <div> - qui sera ici la première sous-balise!
first_div.find_next()

In [ ]:
# Cherchons maintenant toutes les balises de lien (balises <a>) dans la page.
soup.find_all('a')

In [ ]:
# On voit que les liens internes appartiennent à la classe "lien" ; cherchons ceux-ci :
soup.find_all('a', {'class': 'lien'})

In [ ]:
def clean_text(string):
    """Retire les sauts de ligne, puces et indentations d'une chaîne de caractères donnée."""
    return string.replace('\r', '').replace('\n', '').replace('\t', '').replace('\xa0', '')

In [ ]:
# Extrayons proprement les liens internes au site de l'Insee trouvés sur la page d'accueil.
liens_insee = [
    (clean_text(a.text), 'https://insee.fr' + clean_text(a['href']))
    for a in soup.find_all('a', {'class': 'lien'})
]
liens_insee

In [ ]:
# Quels sont les liens dont le descriptif mentionne le mot "européen" ?
["%s : %s" % a for a in liens_insee if 'européen' in a[0]]

In [ ]:
# Mince, il y a un doublon. Dédupliquons les liens listés, et recommençons.
liens_insee = list(set(liens_insee))
["%s : %s" % a for a in liens_insee if 'européen' in a[0]]

In [ ]:
# Si maintenant on veut scraper ces pages, rien de plus simple :
pages_europe = [
    requests.get(url)
    for description, url in liens_insee
    if 'européen' in description
]

In [ ]:
pages_europe

In [ ]:
def extract_text(response):
    """Fonction rapide d'extraction du texte dans une page requêtée avec Requests."""
    soup = BeautifulSoup(response.text, 'html.parser')
    text = '. '.join(
        clean_text(paragraph.text)
        for paragraph in soup.find_all('p')
    )
    return text.replace(' .', '')

In [ ]:
# Ce n'est pas parfait, mais c'est un début.
list(map(extract_text, pages_europe))

### Résumé des éléments fondamentaux de BeautifulSoup :

* Importer la classe BeautifulSoup : `from bs4 import BeautifulSoup`
* Parser un texte html et l'assigner à une variable `soup` : `soup = BeautifulSoup(html_string, 'html.parser')`
* Parser le html obtenu via une requête `Requests.get` réussie : `soup = BeautifulSoup(response.text, 'html.parser')`


* Recherche d'une ou plusieurs balises dans le document parsé (ici noté `soup`):
  * `soup.find()` : retourne la première balise répondant aux critères spécifiés.
  * `soup.find_all()` : retourne la liste de toutes les balises répondant aux critères.
  
  
* Les critères de recherchent peuvent porter sur :
  * Le type de balise (premier argument des fonctions de recherche).
    * un seul type : `soup.find('div')`
    * ou plusieurs : `soup.find(['h1', 'h2'])`
  * Un ou plusieurs attributs de la ou des balises recherchées.
    * les attributs comme `id` ou `href` peuvent être passés comme arguments : `soup.find(id='toto')`
    * certains ne passent pas ainsi (notamment `class`, qui est un mot-clef réservé de Python) ;
      on passe alors un dictionaire : `soup.find_all({'class': 'toto'})`.
  * Le texte contenu dans la ou les balises recherchées.
    * `soup.find(text="Fil d'actualités")`
    * Pour le texte (et parfois pour les attributs), on aura tendance à vouloir des critères plus
      souples qu'une adéquation stricte : on utilisera alors les expressions régulières, comme
      nous le montrerons par la suite.


* Accès aux éléments associés à une balise (ici notée `tag`):
  * `tag.name` : type de balise.
  * `tag.attrs` : attributs de la balise, sous forme de dictionaire.
  * `tag.get()` : accès à un attribut, avec valeur par défaut spécifiable s'il n'existe pas (`̀tag.get('id', '')`).
  * `tag.text` : accès aux textes contenus dans la balise et ses sous-balises, concaténés.
  * `tag.get_text(separator=' ')` : similaire au précédent, en spécifiant un séparateur entre textes concaténés.
  * `tag.contents` : liste des sous-balises.


* Recherche d'une ou plusieurs balises relativement à une autre balise (ici notée `tag`):
  * ̀`tag.find_next()` : retourne la prochaine balise répondant aux critères spécifiés.
  * `tag.find_next_sibling()` : retourne la prochaine balise du même type que `tag` et répondant aux critères spécifiés.
  * `tag.find_all_next()` : retourne toutes les balises postérieures à la présente et répondant aux critères spécifiés.
  * `tag.find_next_siblings()` : retourne toutes les balises postérieures de même type et répondant aux critères
     spécifiés.
      * _Nota Bene_ : en remplaçant `next` par `previous` pour chacune des méthodes précédentes, on accès aux balises
        antérieures à `tag` selon la même logique.
  * `tag.find_parent()` : retourne la plus proche balise englobante répondant aux critères spécifiés.
  * `tag.find_parents()` : liste les balises englobant la présente et répondant aux critères spécifiés.

### 2.2 Les expressions régulières

* Les expressions régulières (aussi appelées expressions rationnelles et souvent abrégées _regex_ ou _regexp_)
  sont une chaîne de caractères définissant, selon une syntaxe formelle précise, un ensemble de chaînes de
  caractères possibles. Il s'agit donc d'un outil souple et efficace pour (notamment) faire des recherches
  dans un texte ou le transformer.


* On ne fera pas ici un cours sur les expressions regulières (ce n'est pas le sujet) ; l'excellent site
  <a href="http://www.regexr.com/">regexr</a> peut être utilisé pour s'y familiariser, voire pour effectuer
  des tests même une fois la syntaxe maîtrisée.


* Les expressions régulières sont implémentées dans Python comme dans R, quoi que ce dernier langage ajoute des
  spécificités demandant de bien lire la documentation (`?regex`), notamment concernant les différents modes de
  compatibilité des expressions.
  
  
* La librairie standard `re` permet d'utiliser les expressions régulières en Python :
  * `re.search(pattern, text)` : rechercher la première occurence d'un pattern dans un texte.
  * `re.findall(pattern, text)` : rechercher toutes les occurences d'une pattern dans un texte.
  * `re.compile(pattern)` : pré-compiler un pattern (utile dans certains contextes, comme on le verra).
  * `re.sub(pattern, remplacement, text)` : rechercher et remplacer un pattern par une chaîne de caractères
     dans un texte.
  * `re.split(pattern, text)` : découper un texte (similairement à `str.split()`) le long d'un pattern.
 
 
* Il peut être pratique d'utiliser des expressions régulières sur du html. Cependant, c'est très rarement une bonne
  idée de parser du html avec des expressions régulières, car à la moindre faute de syntaxe (ou subtilité syntaxique
  non anticipée), on va se planter. Nous allons cependant voir que les expressions régulières peuvent être utilisées
  de manière complémentaire avec BeautifulSoup.

In [ ]:
# Identifions toutes les balises <a> dans la page d'accueil de l'Insee. Cela fonctionne, même si c'est mal.
import re

balises_a = re.findall('<a .+?</a>', response.text)
balises_a

In [ ]:
# Similairement à ce que l'on a déjà fait, identifions les liens internes au site de l'Insee, avec des regex.
liens_insee_regex = [
    (
        clean_text(re.search('(?<=>).+?(?=<)', a).group()),
        'https://insee.fr' + clean_text(re.search('(?<=href=").+?(?=")', a).group())
    )
    for a in balises_a if 'class="lien"' in a
]
# Sans oublier de dédoublonner.
liens_insee_regex = list(set(liens_insee_regex))
# Est-ce bien le même résultat ? Non ; on a manqué des liens, semble-t-il.
liens_insee == liens_insee_regex

#### Alors, comment _bien_ utiliser les expressions régulières ?

* On peut _parfois_ chercher un élément directement dans le html avec les expressions régulières, si l'on est
  vraiment sûr que ce sera le bon et que BeautifulSoup ne le permet pas (ou pas aussi facilement). Mais cela
  reste une pratique risquée et peu conseillée.


* On peut sutout utiliser les expressions régulières **conjointement à BeautifulSoup** :
  * Pour spécifier des critères de recherche au niveau de BeautifulSoup : une expression régulière
    compilée avec `re.compile` peut être passée en critère de recherche, pour la valeur d'un attribut
    ou du texte contenu dans une balise.
  * Pour nettoyer et/ou filtrer un contenu déjà sélectionné à l'aide de BeautifulSoup : il est parfois
    plus efficace de mettre un critère de recherche peu restrictif, et de filtrer les résultats à l'aide
    d'une expression régulière (compiler une expression et rechercher à son aide peut s'avérer coûteux).
    Les expressions regulières sont aussi un excellent outil pour nettoyer du texte, ou en extraire une
    sous-partie !

In [ ]:
# Cherchons les liens pointant sur un autre site que celui de l'Insee.
# Ces liens commencent par 'http' (le protocole) car ils sont absolus, pas relatifs.
# NB : on peut avoir un lien "interne" absolu, le critère retenu ici n'est pas le plus efficace !
liens_externes = [
    clean_text(a['href']) for a in soup.find_all('a', href=re.compile('^http'))
]
liens_externes

In [ ]:
# Plutôt que de compiler l'expression régulière, on peut faire un filtre post-sélection.
# C'est bien aussi, et surtout plus rapide (dans le cas présent).
liens_externes = [
    clean_text(a['href']) for a in soup.find_all('a')
    if re.search('^http', a.get('href', '')) is not None
]
liens_externes

In [ ]:
# Mais si l'on réfléchit, dans cet exemple précis, les regex sont en fait inutiles...
liens_externes = [
        clean_text(a['href']) for a in soup.find_all('a')
        if a.get('href', '').startswith('http')
    ]
liens_externes

In [ ]:
# Testons les temps d'exécution des trois solutions précédentes.
from timeit import timeit

def solution_regex_compil():
    liens_externes = [
    clean_text(a['href']) for a in soup.find_all('a', href=re.compile('^http'))
]
def solution_regex_filtre():
    liens_externes = [
    clean_text(a['href']) for a in soup.find_all('a')
    if re.search('^http', a.get('href', '')) is not None
]
def solution_sans_regex():
    liens_externes = [
        clean_text(a['href']) for a in soup.find_all('a')
        if a.get('href', '').startswith('http')
    ]
    
for function in [solution_regex_compil, solution_regex_filtre, solution_sans_regex]:
    temps = timeit(function, number=1000)
    print('%s : %s' % (function.__name__, temps))

<center>Selon les tâches, la combinaison d'outils est donc à adapter !</center>

### 2.3 Identifier efficacement les cibles

* Le degré souhaitable d'identification de la structure de la page va différer selon la tâche que l'on résout.
  * Un premier élément est bien sûr la nature de l'information reccueillie : chercher des liens ou images est
    globalement plus simple que chercher du texte, car les balises les abritant sont très spécifiques. Il peut
    alors rester à définir des filtres adéquats. Les points suivants portent surtout sur des cas où l'on cherche
    à récolter une information plus spécifique (texte ou champs spécifiques, par exemple).

  * Dans le cas d'un _scraping_ portant sur une diversité de sources (_scraping_ dit "générique"), on va surtout
    réfléchir au type de balises que l'on vise, aux filtres éventuels que l'on peut appliquer sur les résultats
    (présence de certains mots-clefs dans un texte par exemple) et à un certain mode de structuration des données
    permettant de rendre compte de ce qu'elles sont à l'origine.
    * _Par exemple, si l'on récupère des articles de presse sur de nombreux sites différents, on peut chercher
      à identifier les sections correspondant aux menus et aux commentaires (pour les éliminer), pour se focaliser
      sur ce qui semble être le contenu principal (selon le nom des balises l'encadrant, la présence de titres,
      la longueur du texte...)._
  * Dans le cas d'un _scraping_ portant sur une source précise (ou plusieurs sources distinctes faisant l'objet
    de traitements d'extraction différenciés), on va chercher à acquérir une information plus fine sur la structuration
    des pages ciblées. Pour cela, on fait une espèce de _reverse engineering_ à partir de l'observation de pages
    de test, pour définir des règles précises d'extraction de l'information.


* Afin d'étudier la structure des pages sur un site, plusieurs outils peuvent être très pratiques :
    * Un outil déjà mentionné est l'inspecteur d'éléments de Firefox (dont des équivalents existent
      pour certains autres navigateurs, comme Chromium). Il permet de visualiser le code source de
      la page actuellement rendue, de l'altérer pour constater des changements induits "en direct",
      et surtout de trouver la portion de code associée à un élément visuel d'un simple clic droit
      ("inspecter l'élément").
    
    * Plus généralement, regarder le code source d'une page (ctrl + u dans un navigateur web) est toujours
      une bonne idée. Il s'agit d'en comprendre l'organisation globale, de comparer la page rendue à son code
      source pour comprendre quelle section fait quoi, et d'identifier où se trouvent les éléments que l'on
      souhaite extraire, à grands renforts de ctrl + f (rechercher) si besoin.
      
    * Enfin, il ne pas oublier de regarder s'il n'y as pas "plus" ou "mieux" que ce que l'on cherche initialement,
      à d'autres endroits du code source que ceux rendus visuellement. En particulier, des scripts sont susceptibles
      de comporter des variables structurant au format JSON une information par la suite répartie dans des structures
      visuelles lors du rendu de la page. Ces variables, écrites en clair, sont généralement faciles à récupérer par
      une combinaison BeautifulSoup / RegExp, et peuvent être parsée à l'aide de la librairie standard `json`, qui
      va transformer la chaîne de caractères obtenue en une structure à base de dictionaires et de listes. Et si
      `json` ne fonctionne pas, on peut aussi essayer `yaml` (librairie tierce `pyyaml`).

#### Démonstration : scrapons des offres d'emploi.

* On part de l'url des pages de recherche d'offres d'emploi.
* On trouve le paramètre permettant d'itérer sur les pages de recherche.
* On identifie une caractéristique des liens d'offres d'emploi, on en déduit une fonction pour identifier ces liens.
* On étudie maintenant une page d'offre d'emploi. On trouve ce que l'on a envie d'extraire dans le code source.
* On galère un peu avec json / yaml, on finit par trouver, on écrit la fonction de parsing magique.
* On réfléchit cinq minutes au format de stockage des données. En général, csv c'est très bien.
* Et hop, il n'y a plus qu'à organiser les "bonnes" itérations pour scraper !
* (Si on veut faire propre, on formalise le tout, on gère les erreurs, etc.)

In [ ]:
# Ici, faire de la magie en suivant la to-do list précédente.

## 3. Compléments et ouvertures

### 3.1 Quelques éléments de législation et de bonnes pratiques

* La législation sur la _scraping_ est globalement floue. Cette pratique est globalement tolérée, mais un site
  peut s'y opposer (conditions d'utilisation, dont le champ d'application est cependant incertain), et en tout
  cas (tenter de) la réguler en spécifiant des règles d'accès _via_ le fichier `robots.txt`.
  
  
* (Presque) tous les sites sont dotés d'un fichier `robots.txt`, à l'adresse `http(s)://<nom_de_domaine>/robots.txt`.
  Celui-ci spécifie des règles d'interdiction de pages ou sections de l'arborescence du site, pouvant être différenciés
  par agent d'utilisation. Il peut aussi spécifier des délais (en secondes) à respecter entre deux requêtes.
  Globalement, c'est bien de respecter ces règles, même si on peut avoir envie de les négliger dans le cadre d'un
  hackathon...


* Pour un _scraping_ visant à soutenir une production statistique, il faut contacter le site pour lui laisser la
  possibilité de s'y opposer. Indiquer son identité est aussi une bonne chose :<br/>
  `headers = {'user-agent': "Coucou, c'est l'Insee."}`<br/>
  `response = requests.get(url, headers=headers)`

### 3.2 Comment optimiser le coût temporel d'un _scraping_ ?

Le _scraping_ de nombreuses pages prend du temps, surtout si on respecte un délai entre les requêtes en accord
avec le `robots.txt` du site visé. Le problème est dit I/O-blocant, c'est-à-dire que le coût provient surtout
du délai nécessaire à l'envoi d'une requête HTTP et à la réception de la réponse ; parser du html est généralement
assez rapide. On a cependant envie d'envoyer des requêtes _aussi vite que possible / permis_, plutôt que de "perdre"
le petit peu de temps que prend l'extraction d'information. Une solution peut alors être de distribuer les opérations.

Ci-dessous, un exemple de structure permettant de répondre à ce problème.

In [ ]:
import multiprocessing
import time

def download(url, headers=None):
    """Requête une url donnée et retourne la réponse."""
    response = requests.get(url, headers=headers)

def parse(response):
    """Extrait l'information voulue d'une page html."""
    if response.status_code != 200:
        return None
    soup = BeautifulSoup(response, 'html.parser')
    infos = {}
    # Extraction d'information ici, stockées dans `infos`.
    return infos

def process(url, headers, queue):
    response = download(url, headers)
    results = parse(response)
    if results is not None:
        queue.put(results)

def main(urls_list, delay=1, headers=None, pool_size=2):
    """Télécharge et parse une liste d'urls, en respectant un délai donné entre requêtes."""
    queue = multiprocessing.Manager().Queue()
    with multiprocessing.Pool(pool_size) as pool:
        # Toutes les <delay> secondes, on lance le traitement d'une nouvelle url.
        for url in urls_list:
            pool.apply_aync(func=process, args=(url, headers, queue))  # apply_async effectue la tâche en arrière-plan.
            time.sleep(delay)
            # Ici on s'assure de ne pas créer de file d'attente, qui pourrait entraîner le franchissement du délai.
            while len(pool._cache) >= pool_size:
                time.sleep(.1)
        # Une fois toutes les url soumises, on attend que toutes les opérations de traitement soient finies.
        pool.close()
        pool.join()
    # Une fois toutes les urls traitées, on récupère les résultats :
    results = []
    while queue.qsize() > 0:
        results.append(queue.get())
    return results

Notes :
* A la solution proposée ci-dessus, on peut préférer un système qui lance les requêtes une par une et s'assure
  simplement que le temps d'attente est bien respecté (si la connexion est rapide ou si le délai est "long",
  on ne peut pas optimiser au-delà).
  
  
* Si on veut travailler sur une liste d'urls générée dynamiquement lors de l'analyse des pages initialement visées,
  on peut ajouter un autre système de queue : le parseur y place (`queue.put`) les urls trouvées, tandis que le
  gestionnaire de téléchargements en tire (`queue.get`) les résultats. Il suffit de définir un "signal d'arrêt"
  (e.g. `None`), i.e. un élément qui, lorsqu'il est reçu, signifie que l'on a atteint le "bout" de la queue. Sinon,
  on peut mettre un "simple" _timeout_, i.e. temps d'expiration de l'instruction d'attente.

### 3.3 Comment collecter autre chose que du html ?

* Pour toute ressource au format textuel (fichier css, csv, txt, json...), on peut la requêter directement
  et l'écrire dans un fichier.

In [ ]:
# GeoJson "actions contre l'isolement des personnes âgées".
url = 'https://geo.data.gouv.fr/api/geogw/services/5779810963f06a3a8e81541b/feature-types/C1619/download?format=GeoJSON&projection=WGS84'
response = requests.get(url)

In [ ]:
with open('actions_isolement_personnes_agees.json', 'w') as json_file:  # mode 'w' pour 'write'
    json_file.write(response.text)

* Pour d'autres formats (pdf, xls...), on requête également directement, mais plutôt que d'écrire le texte
  déjà décodé, on stocke la version encodée du contenu téléchargé (et le lecteur de document déchiffrera).
  L'accès à l'information encodée se fait _via_ `response.content`.

In [ ]:
# Pdf "guide de référence Naf v2"
url = 'https://insee.fr/fr/statistiques/fichier/2406147/guide_naf_cpf_rev_2.pdf'
response = requests.get(url)

In [ ]:
with open(url.rsplit('/', 1)[1], mode='wb') as pdf:  # mode 'wb' pour 'write binary'
    pdf.write(response.content)

* Cette dernière solution marche aussi pour les images.

In [ ]:
# Logo de l'Insee
url = 'https://insee.fr/static/img/logo_com_externe_semi_bold.png'
response = requests.get(url)

In [ ]:
with open('logo.png', 'wb') as png:
    png.write(response.content)

In [ ]:
# Visualisons le logo précédemment stocké.
from IPython.display import Image
Image('logo.png')

### 3.4 Comment gérer les cas où JavaScript nous met des bâtons dans les roues ?

[En construction]

* Explication : boutons JavaScript, requêtes asynchrones (Ajax).
* Dans certains cas : la requête touche à une ressource externe (e.g. un json),
  dont on peut récupérer l'adresse pour la requêter.
* Dans d'autres cas : on peut faire des requêtes POST pour déclencher un évènement, s'authentifier, etc.
* Sinon : on va émuler un navigateur web, avec Selenium.

### 3.5 Outil supplémentaire

* `scrapy` : librairie tierce permettant de mettre en place des crawlers / scrapers.
  * fonctionalités déjà existantes.
  * syntaxe spécifique à cette librairie pour parser le html (mais possibilité de lui substituer BeautifulSoup).